# 스칼라 필드 인덱스
- Milvus에서 스칼라 인덱스는 기존 데이터베이스 인덱스와 유사하게 특정 비벡터 필드 값을 기준으로 메타필터링 속도를 높이는 데 사용
- 이 가이드에서는 정수, 문자열 등과 같은 필드에 대한 스칼라 인덱스를 만들고 구성하는 방법을 안내함

참고자료: https://milvus.io/docs/ko/v2.5.x/index-scalar-fields.md

## 스칼라 인덱싱의 유형
- 자동 인덱싱: Milvus는 스칼라 필드의 데이터 유형에 따라 인덱스 유형을 자동으로 결정, 특정 인덱스 유형을 제어할 필요가 없는 경우 적합
- 사용자 지정 인덱싱: 반전 인덱스 또는 비트맵 인덱스와 같은 정확한 인덱스 유형을 지정

## 자동 인덱싱
- 자동 인덱싱을 사용하려면 인덱스 유형 매개변수(add_index())에서 인덱스 유형 매개변수를 생략하면 Milvus가 스칼리 필드 유형을 기반으로 인덱스 유형을 유추

In [12]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("test_scalar_index"):
    client.drop_collection("test_scalar_index")

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=False,
)

schema.add_field(field_name="index", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="scalar_1", datatype=DataType.INT64)
schema.add_field(field_name="scalar_2", datatype=DataType.INT32)

client.create_collection(
    collection_name="test_scalar_index",
    schema=schema
)

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="scalar_1",
    index_type="",
    index_name="default_index"
)

client.create_index(
    collection_name="test_scalar_index",
    index_params=index_params
)

In [13]:
client.describe_index(
    collection_name="test_scalar_index",
    index_name="default_index"
)

{'field_name': 'scalar_1',
 'index_name': 'default_index',
 'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

## 사용자 정의 인덱싱
- 인덱스 유형 매개변수를 사용하여 특정 인덱스 유형을 지정 add_index()
- 매서드 및 매개변수
    - prepare_index_params()
        - IndexParams 객체 준비
    - add_index()
        - IndexParams 객체에 인덱스 구성을 추가
            - field_name(문자열): 인덱싱할 스칼라 필드의 이름
            - index_type(문자열): 생성할 스칼라 인덱스의 유형, 암시적 인덱싱의 경우 이 매개변수를 비워두거나 생략
                - INVERTED: (권장) 반전 인덱스는 토큰화된 모든 단어가 알파벳 순으로 정렬된 용어 사전으로 구성됨
                - 비트맵: 필드에 있는 모든 고유 값의 비트맵을 저장하는 인덱스 유형
                - STL_SORT: 표준 템플릿 라이브러리 정렬 알고리즘을 사용하여 스칼리 필드를 정렬, 숫자 필드만 지원(INT8, INT16, INT32, INT64, FLOAT, DOUBLE)
                - 트라이: 빠른 접두사 검색 및 검색을 위한 트리 데이터 구조, VARCHAR 필드 지원
                - index_name(문자열): 생성할 스칼라 인덱스의 이름, 각 스칼라 필드는 하나의 인덱스를 지원
    - create_index()
        - collection_name(문자열): 인덱스가 생성될 컬렉션의 이름
        - index_params: 인덱스 구성을 포함하는 IndexParams 객체

In [ ]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="scalar_2",
    index_type="INVERTED",
    index_name="inverted_index"
)

client.create_index(
    collection_name="test_scalar_index",
    index_params=index_params
)

## 결과 확인
- list_indexes() 메서드를 사용하여 스칼라 인덱스 생성 확인

In [15]:
client.list_indexes(
    collection_name="test_scalar_index"
)

['default_index', 'inverted_index']